In [17]:
import pandas as pd
from preprocess.data_prep import clean_weather_data
from preprocess.labels import  classify_weather
from preprocess.features import build_features


In [18]:
data = pd.read_csv('../data/raw/JFK Airport Weather Data.csv')
data.head()

,STATION,NAME,DATE,ACMH,ACSH,AWND,FMTM,PGTM,PRCP,SNOW,...,WT13,WT14,WT15,WT16,WT17,WT18,WT19,WT21,WT22,WV01
0,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",1960-01-01,NaN,NaN,NaN,NaN,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",1960-01-02,NaN,NaN,NaN,NaN,NaN,0.08,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",1960-01-03,NaN,NaN,NaN,NaN,NaN,0.68,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",1960-01-04,NaN,NaN,NaN,NaN,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",1960-01-05,NaN,NaN,NaN,NaN,NaN,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
cleaned_data = clean_weather_data(data)
cleaned_data["weather_type"] = cleaned_data.apply(classify_weather, axis=1)
cleaned_data["dayofyear"] = cleaned_data["DATE"].dt.dayofyear
cleaned_data["month"] = cleaned_data["DATE"].dt.month
cleaned_data = build_features(cleaned_data)

In [12]:
regression_columns = [
    "DATE",
    "TMAX",        # target
    "TMIN",
    "TAVG",
    "PRCP",
    "SNOW",
    "AWND",
    "TMAX_lag_1",
    "TMAX_roll_7",
    "dayofyear",
    "month"
]
classification_columns = [
    "DATE",
    "weather_type",  # target
    "PRCP",
    "SNOW",
    "TAVG",
    "TMIN",
    "WT01", "WT02",
    "WT03", "WT06", "WT07", "WT11",
    "PRCP_roll_3",
    "dayofyear",
    "month"
]

In [13]:
classification_data = cleaned_data[classification_columns]

regression_data = cleaned_data[regression_columns]

In [15]:
regression_data.head()

,DATE,TMAX,TMIN,TAVG,PRCP,SNOW,AWND,TMAX_lag_1,TMAX_roll_7,dayofyear,month
0,1960-01-01,41.0,26.0,33.5,0.00,0.0,10.51,NaN,NaN,1,1
1,1960-01-02,45.0,24.0,34.5,0.08,0.0,10.51,41.0,NaN,2,1
2,1960-01-03,51.0,38.0,44.5,0.68,0.0,10.51,45.0,NaN,3,1
3,1960-01-04,40.0,34.0,37.0,0.00,0.0,10.51,51.0,NaN,4,1
4,1960-01-05,38.0,31.0,34.5,0.00,0.0,10.51,40.0,NaN,5,1


In [16]:
classification_data.head()

,DATE,weather_type,PRCP,SNOW,TAVG,TMIN,WT01,WT02,WT03,WT06,WT07,WT11,PRCP_roll_3,dayofyear,month
0,1960-01-01,Sunny,0.00,0.0,33.5,26.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1,1
1,1960-01-02,Rainy,0.08,0.0,34.5,24.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2,1
2,1960-01-03,Rainy,0.68,0.0,44.5,38.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3,1
3,1960-01-04,Sunny,0.00,0.0,37.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.253333,4,1
4,1960-01-05,Sunny,0.00,0.0,34.5,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.253333,5,1


In [24]:
classification_data.to_csv('../data/processed/classification_data.csv')
regression_data.to_csv('../data/processed/regression_data.csv')

In [25]:
df1 = pd.read_csv('../data/processed/regression_data.csv',index_col=0)
df1

,DATE,TMAX,TMIN,TAVG,PRCP,SNOW,AWND,TMAX_lag_1,TMAX_roll_7,dayofyear,month
0,1960-01-01,41.0,26.0,33.5,0.00,0.0,10.51,NaN,NaN,1,1
1,1960-01-02,45.0,24.0,34.5,0.08,0.0,10.51,41.0,NaN,2,1
2,1960-01-03,51.0,38.0,44.5,0.68,0.0,10.51,45.0,NaN,3,1
3,1960-01-04,40.0,34.0,37.0,0.00,0.0,10.51,51.0,NaN,4,1
4,1960-01-05,38.0,31.0,34.5,0.00,0.0,10.51,40.0,NaN,5,1
...,...,...,...,...,...,...,...,...,...,...,...
20084,2014-12-27,54.0,37.0,44.0,0.00,0.0,6.26,51.0,47.857143,361,12
20085,2014-12-28,54.0,42.0,47.0,0.05,0.0,10.51,54.0,50.714286,362,12
20086,2014-12-29,46.0,35.0,43.0,0.00,0.0,10.96,54.0,53.000000,363,12
20087,2014-12-30,36.0,28.0,34.0,0.00,0.0,9.40,46.0,52.714286,364,12
